In [16]:
# Dataset : https://www.kaggle.com/datasets/gauravduttakiit/ants-bees?resource=download
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms,models
from torch.utils.data import DataLoader

In [17]:
transforms={'train':transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]),
    'val':transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomResizedCrop(128),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
}

In [18]:
data_dir='data/archive/hymenoptera_data'
#image_datasets={x:datasets.ImageFolder(os.path.join(data_dir,x),transform=transforms[x]) for x in ['train','val']}


In [19]:
image_datasets={}
image_datasets['train']=datasets.ImageFolder(os.path.join(data_dir,'train'),transform=transforms['train'])
image_datasets['val']=datasets.ImageFolder(os.path.join(data_dir,'val'),transform=transforms['val'])

In [20]:
data_loader={}
data_loader['train']=DataLoader(image_datasets['train'],batch_size=32,shuffle=True)
data_loader['val']=DataLoader(image_datasets['val'],batch_size=32,shuffle=True)

In [21]:
model=models.resnet18(pretrained=True)
#print(model)
model_ftrs=model.fc.in_features
model.fc = nn.Linear(model_ftrs,out_features=2)
#model.fc.ft1=nn.Linear(model_ftrs,out_features=256)
#model.fc.ft2=nn.Linear(256,out_features=2)
#model.ft.ft3=nn.Linear(64,out_features=16)
#model.ft.ft4=nn.Linear(16,out_features=2)

criterian=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.fc.parameters(),lr=0.0001)


In [22]:
for epoch in range(10):
    for input,label in data_loader['train']:
        output=model(input)
        loss=criterian(output,label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(loss.item())


0.7121381759643555
0.556524395942688
0.7504733800888062
0.5157186985015869
0.5544464588165283
0.458655446767807
0.5447261929512024
0.5195502042770386
0.4784887731075287
0.6389667391777039


In [23]:
model.eval()
with torch.no_grad():
    correct=0
    total=0
    for input,label in data_loader['val']:
        output=model(input)
        _,predicted=torch.max(output.data,1)
        total+=label.size(0)
        correct+=(predicted==label).sum().item()
    print('Accuracy: {}%'.format(100*correct/total))
    print(total, correct)

Accuracy: 76.47058823529412%
153 117
